# LlamaIndex Bottoms-Up Development - LLMs and Prompts
This notebook walks through testing an LLM using the primary prompt templates used in llama-index.

In [1]:
import openai
import os

openai.api_key = ""
os.environ["OPENAI_API_KEY"] = ""

## Setup
In this section, we load a test document, create an LLM, and copy prompts from llama-index to test with.

First, let's load a quick document to test with. Right now, we will just load it as plain text, but we can do other operations later!

In [3]:
with open("input.txt", "r") as f:
    text = f.read()

Next, we create our LLM!

In [4]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

LlamaIndex uses some simple templates under the hood for answering queries -- mainly a `text_qa_template` for obtaining initial answers, and a `refine_template` for refining an existing answer when all the text does not fit into one LLM call.

Let's copy the default templates, and test out our LLM with a few questions.

In [7]:
from llama_index.core import PromptTemplate

text_qa_template = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

refine_template = (
    "We have the opportunity to refine the original answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question: {query_str}. "
    "If the context isn't useful, output the original answer again.\n"
    "Original Answer: {existing_answer}"
)

text_qa_template = PromptTemplate(text_qa_template)
refine_template = PromptTemplate(refine_template)


Now, lets test a few questions!

## Text QA Template Testing

In [11]:
question = "Explain me about Vulnerabilities and defenses in LLMs."
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

Vulnerabilities in Large Language Models (LLMs) can be categorized into two main groups: AI Model Inherent Vulnerabilities and Non-AI Model Inherent Vulnerabilities. AI Model Inherent Vulnerabilities include issues like data poisoning, backdoor attacks, and training data extraction, while Non-AI Model Inherent Vulnerabilities encompass concerns like remote code execution, prompt injection, and side channels. These vulnerabilities pose threats to both security (e.g., remote code execution attacks) and privacy (e.g., data extraction).

To defend against these vulnerabilities, strategies can be implemented at various stages. During the training phase, defenses may involve cleaning training corpora and optimizing methods. In the inference phase, defenses can include pre-processing instructions, detecting malicious behavior, and post-processing generated outputs. These defense mechanisms aim to enhance the security, robustness, and ethical alignment of LLMs.

It is noted that certain types 

In [9]:
question = "What is data integrity?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

Data integrity refers to the accuracy, consistency, and reliability of data throughout its lifecycle. It ensures that data remains unchanged and is not altered or corrupted in any unauthorized or unintended way. This includes maintaining the completeness and reliability of data, as well as protecting it from unauthorized access, modification, or deletion. In the context of the provided text, data integrity is one of the aspects of data security and privacy that Large Language Models (LLMs) can help ensure by applying techniques such as data confidentiality protection and data traceability.


In [17]:
question = "How do I create an index? Write your answer using only code."
prompt = text_qa_template.format(context_str=text, query_str=question)
response_gen = llm.stream_complete(prompt)
for response in response_gen:
    print(response.delta, end="")

```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)
```

## Refine Template Testing

In [18]:
question = "How do I create an index? Write your answer using only code."
existing_answer = """To create an index using LlamaIndex, you need to follow these steps:

1. Download the LlamaIndex repository by cloning it from GitHub.
2. Navigate to the `examples/paul_graham_essay` folder in the cloned repository.
3. Create a new Python file and import the necessary modules: `VectorStoreIndex` and `SimpleDirectoryReader`.
4. Load the documents from the `data` folder using `SimpleDirectoryReader('data').load_data()`.
5. Build the index using `VectorStoreIndex.from_documents(documents)`.
6. To persist the index to disk, use `index.storage_context.persist()`.
7. To reload the index from disk, use the `StorageContext` and `load_index_from_storage` functions.

Note: This answer assumes that you have already installed LlamaIndex and have the necessary dependencies."""
prompt = refine_template.format(context_msg=text, query_str=question, existing_answer=existing_answer)
response = llm.complete(prompt)
print(response.text)

To create an index using LlamaIndex, follow these steps:

```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader

# Load the documents from the 'data' folder
documents = SimpleDirectoryReader('data').load_data()

# Build the index
index = VectorStoreIndex.from_documents(documents)

# Persist the index to disk
index.storage_context.persist()

# Reload the index from disk
from llama_index import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context)
```

Make sure you have installed LlamaIndex and have the necessary dependencies.


### Chat Example
The LLM also has a `chat` method that takes in a list of messages, to simulate a chat session. 

In [14]:
from llama_index.core.llms import ChatMessage

chat_history = [
    ChatMessage(role="system", content="You are a helpful QA chatbot that can answer questions about llama-index."),
    ChatMessage(role="user", content="How do I create an index?"),
]

response = llm.chat(chat_history)
print(response.message)

assistant: To create an index, you typically need to follow these steps:

1. Determine the purpose of the index: Decide what you want the index to represent or measure.

2. Select the components: Choose the assets, securities, or items that will be included in the index.

3. Assign weights: Decide on the weighting methodology for the components of the index (e.g., market capitalization-weighted, equal-weighted, price-weighted).

4. Calculate the index value: Calculate the index value based on the components and their weights.

5. Rebalance the index: Regularly review and adjust the components and weights of the index to ensure it remains relevant and representative.

6. Publish the index: Make the index available to users through various channels such as financial websites, data providers, or index providers.

If you are specifically interested in llama-index, you can refer to the documentation or guidelines provided by llama-index to create an index using their platform.
